In [1]:
from sklearn.ensemble import RandomForestClassifier
import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC

2022-11-07 13:15:33.466595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-07 13:15:33.570655: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-07 13:15:33.570672: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-07 13:15:33.591821: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-07 13:15:34.139018: W tensorflow/stream_executor/platform/de

In [2]:
AUC_ROC=[]
model=[]
def list_auc(list_f,list_m):
    main_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_main_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            main_all_feature.insert(loc=len(main_all_feature.columns),column=m+feature,value=pred1)


    external_all_feature=pd.DataFrame()
    for feature in list_f:
        for m in list_m:
            read_main=pd.read_csv('pred_'+m+'_external_set_'+feature+'.csv')
            df_main=pd.DataFrame(read_main)
            pred1=[]
            for p in df_main['pred']:
                if p<=0.5:
                    p=0
                else:
                    p=1
                pred1.append(p)
            external_all_feature.insert(loc=len(external_all_feature.columns),column=m+feature,value=pred1)
    
    read_m = pd.read_csv("main_set_plus_AtomPairFP.csv")
    df_m = pd.DataFrame(read_m)
    a1 = df_m['active']
    c1=[]
    for b1 in a1:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_m=pd.DataFrame()
    y_m.insert(loc=0, column='active', value=a1)
    y_m.insert(loc=1, column='inactive', value=c1)
    X_m=main_all_feature

    read_e = pd.read_csv("external_set_plus_AtomPairFP.csv")
    df_e = pd.DataFrame(read_e)
    a2 = df_e['active']
    c1=[]
    for b1 in a2:
        if b1==1:
            b1=0
        else:
            b1=1
        c1.append(b1)

    y_e=pd.DataFrame()
    y_e.insert(loc=0, column='active', value=a2)
    y_e.insert(loc=1, column='inactive', value=c1)
    X_e=external_all_feature

    batch_size = 8192
    num_classes = 2
    model = Sequential()
    model.add(Dense(800,
                    activation='relu',
                    input_dim=len(X_m.columns),
                    kernel_initializer = initializers.RandomNormal(stddev=0.02),
                    bias_initializer = initializers.RandomNormal(mean=1.),
                    kernel_regularizer = regularizers.l2(1e-1)))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    opt = keras.optimizers.Adam(learning_rate=0.001)

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=opt,
                  metrics=['AUC'])
    model.fit(X_m, y_m,
              batch_size=batch_size,
              epochs=100)
    score = model.evaluate(X_e, y_e, verbose=0)
    print(score[1])
    AUC_ROC.append(score[1])
    model=''

In [3]:
m='all_model'
a=['AtomPairFP','Charge',
       'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100


2022-11-07 13:15:38.377433: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-07 13:15:38.378732: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2022-11-07 13:15:38.378848: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcurand.so.10'; dlerror: libcurand.so.10: cannot open shared object file: No such file or directory
2022-11-07 13:15:38.378950: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2022-11-07 13:15:38.379048: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could no

1/1 [==============================] - 0s 394ms/step - loss: 2.3745 - auc: 0.3044
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5020 - auc: 0.7072
Epoch 3/100
1/1 [==============================] - 0s 24ms/step - loss: 1.4203 - auc: 0.8326
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.5303 - auc: 0.8499
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.5739 - auc: 0.8624
Epoch 6/100
1/1 [==============================] - 0s 28ms/step - loss: 1.5512 - auc: 0.8690
Epoch 7/100
1/1 [==============================] - 0s 37ms/step - loss: 1.4695 - auc: 0.8786
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.3712 - auc: 0.8832
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2211 - auc: 0.8929
Epoch 10/100
1/1 [==============================] - 0s 29ms/step - loss: 1.0732 - auc: 0.9061
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - loss: 0.9753 

1/1 [==============================] - 0s 32ms/step - loss: 0.2406 - auc: 0.9918
Epoch 89/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2505 - auc: 0.9901
Epoch 90/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2537 - auc: 0.9892
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2460 - auc: 0.9907
Epoch 92/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2415 - auc: 0.9910
Epoch 93/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2383 - auc: 0.9918
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2530 - auc: 0.9884
Epoch 95/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2381 - auc: 0.9918
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2443 - auc: 0.9903
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2440 - auc: 0.9904
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [4]:
m='remove_AtomPairFP'
a=['Charge',
      'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 1.4493 - auc: 0.6985
Epoch 2/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4951 - auc: 0.7988
Epoch 3/100
1/1 [==============================] - 0s 28ms/step - loss: 1.3663 - auc: 0.8157
Epoch 4/100
1/1 [==============================] - 0s 36ms/step - loss: 1.1983 - auc: 0.8289
Epoch 5/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1269 - auc: 0.8207
Epoch 6/100
1/1 [==============================] - 0s 35ms/step - loss: 1.1121 - auc: 0.8068
Epoch 7/100
1/1 [==============================] - 0s 38ms/step - loss: 1.0424 - auc: 0.8334
Epoch 8/100
1/1 [==============================] - 0s 28ms/step - loss: 0.9463 - auc: 0.8789
Epoch 9/100
1/1 [==============================] - 0s 27ms/step - loss: 0.8826 - auc: 0.9068
Epoch 10/100
1/1 [==============================] - 0s 33ms/step - loss: 0.8570 - auc: 0.9170
Epoch 11/100
1/1 [==============================] - 0s 27ms/step - l

1/1 [==============================] - 0s 32ms/step - loss: 0.2696 - auc: 0.9876
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2637 - auc: 0.9887
Epoch 90/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2704 - auc: 0.9865
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2741 - auc: 0.9860
Epoch 92/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2677 - auc: 0.9876
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2652 - auc: 0.9880
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2752 - auc: 0.9856
Epoch 95/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2621 - auc: 0.9885
Epoch 96/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2705 - auc: 0.9866
Epoch 97/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2632 - auc: 0.9883
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [5]:
m='remove_Charge'
a=['AtomPairFP',
       'MAP4','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 336ms/step - loss: 2.9130 - auc: 0.2368
Epoch 2/100
1/1 [==============================] - 0s 24ms/step - loss: 1.6775 - auc: 0.5287
Epoch 3/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2345 - auc: 0.8153
Epoch 4/100
1/1 [==============================] - 0s 26ms/step - loss: 1.3110 - auc: 0.8492
Epoch 5/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4050 - auc: 0.8719
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5052 - auc: 0.8755
Epoch 7/100
1/1 [==============================] - 0s 37ms/step - loss: 1.5565 - auc: 0.8699
Epoch 8/100
1/1 [==============================] - 0s 29ms/step - loss: 1.4826 - auc: 0.8845
Epoch 9/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4131 - auc: 0.8883
Epoch 10/100
1/1 [==============================] - 0s 24ms/step - loss: 1.3239 - auc: 0.8936
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 25ms/step - loss: 0.2582 - auc: 0.9900
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2458 - auc: 0.9906
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2486 - auc: 0.9911
Epoch 91/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2548 - auc: 0.9895
Epoch 92/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2481 - auc: 0.9906
Epoch 93/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2544 - auc: 0.9890
Epoch 94/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2508 - auc: 0.9899
Epoch 95/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2525 - auc: 0.9893
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2483 - auc: 0.9901
Epoch 97/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2446 - auc: 0.9907
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [6]:
m='remove_MAP4'
a=['AtomPairFP',
       'Charge','MOE','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 342ms/step - loss: 1.8782 - auc: 0.4125
Epoch 2/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3822 - auc: 0.7469
Epoch 3/100
1/1 [==============================] - 0s 26ms/step - loss: 1.4167 - auc: 0.8124
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 1.4902 - auc: 0.8239
Epoch 5/100
1/1 [==============================] - 0s 23ms/step - loss: 1.4513 - auc: 0.8419
Epoch 6/100
1/1 [==============================] - 0s 27ms/step - loss: 1.3510 - auc: 0.8517
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.2222 - auc: 0.8555
Epoch 8/100
1/1 [==============================] - 0s 27ms/step - loss: 1.1268 - auc: 0.8546
Epoch 9/100
1/1 [==============================] - 0s 31ms/step - loss: 1.0056 - auc: 0.8684
Epoch 10/100
1/1 [==============================] - 0s 26ms/step - loss: 1.0117 - auc: 0.8440
Epoch 11/100
1/1 [==============================] - 0s 31ms/step - l

1/1 [==============================] - 0s 27ms/step - loss: 0.2888 - auc: 0.9843
Epoch 89/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2981 - auc: 0.9818
Epoch 90/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2960 - auc: 0.9821
Epoch 91/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2955 - auc: 0.9820
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2918 - auc: 0.9828
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.3062 - auc: 0.9789
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2882 - auc: 0.9841
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2930 - auc: 0.9834
Epoch 96/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2837 - auc: 0.9850
Epoch 97/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2840 - auc: 0.9849
Epoch 98/100
1/1 [==============================] - 0s 24ms/step - loss: 

In [7]:
m='remove_MOE'
a=['AtomPairFP',
       'Charge','MAP4','MorganFP','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 338ms/step - loss: 1.5389 - auc: 0.6350
Epoch 2/100
1/1 [==============================] - 0s 34ms/step - loss: 1.4552 - auc: 0.7913
Epoch 3/100
1/1 [==============================] - 0s 27ms/step - loss: 1.4099 - auc: 0.8197
Epoch 4/100
1/1 [==============================] - 0s 33ms/step - loss: 1.2570 - auc: 0.8361
Epoch 5/100
1/1 [==============================] - 0s 28ms/step - loss: 1.1435 - auc: 0.8399
Epoch 6/100
1/1 [==============================] - 0s 32ms/step - loss: 1.1150 - auc: 0.8211
Epoch 7/100
1/1 [==============================] - 0s 33ms/step - loss: 1.0580 - auc: 0.8294
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 0.9776 - auc: 0.8615
Epoch 9/100
1/1 [==============================] - 0s 25ms/step - loss: 0.9076 - auc: 0.8925
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 0.8446 - auc: 0.9169
Epoch 11/100
1/1 [==============================] - 0s 32ms/step - l

1/1 [==============================] - 0s 26ms/step - loss: 0.2618 - auc: 0.9878
Epoch 89/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2597 - auc: 0.9891
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2540 - auc: 0.9899
Epoch 91/100
1/1 [==============================] - 0s 21ms/step - loss: 0.2472 - auc: 0.9911
Epoch 92/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2486 - auc: 0.9910
Epoch 93/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2605 - auc: 0.9888
Epoch 94/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2454 - auc: 0.9912
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2550 - auc: 0.9891
Epoch 96/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2517 - auc: 0.9895
Epoch 97/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2507 - auc: 0.9899
Epoch 98/100
1/1 [==============================] - 0s 27ms/step - loss: 

In [8]:
m='remove_MorganFP'
a=['AtomPairFP',
       'Charge','MAP4','MOE','Topology']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 340ms/step - loss: 3.0567 - auc: 0.2348
Epoch 2/100
1/1 [==============================] - 0s 25ms/step - loss: 1.7115 - auc: 0.5007
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2364 - auc: 0.8073
Epoch 4/100
1/1 [==============================] - 0s 34ms/step - loss: 1.2885 - auc: 0.8506
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 1.4301 - auc: 0.8594
Epoch 6/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5314 - auc: 0.8640
Epoch 7/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5537 - auc: 0.8688
Epoch 8/100
1/1 [==============================] - 0s 34ms/step - loss: 1.5433 - auc: 0.8652
Epoch 9/100
1/1 [==============================] - 0s 30ms/step - loss: 1.4601 - auc: 0.8771
Epoch 10/100
1/1 [==============================] - 0s 25ms/step - loss: 1.3710 - auc: 0.8821
Epoch 11/100
1/1 [==============================] - 0s 35ms/step - l

1/1 [==============================] - 0s 30ms/step - loss: 0.2873 - auc: 0.9873
Epoch 89/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2923 - auc: 0.9856
Epoch 90/100
1/1 [==============================] - 0s 22ms/step - loss: 0.2825 - auc: 0.9878
Epoch 91/100
1/1 [==============================] - 0s 29ms/step - loss: 0.2807 - auc: 0.9884
Epoch 92/100
1/1 [==============================] - 0s 24ms/step - loss: 0.2810 - auc: 0.9878
Epoch 93/100
1/1 [==============================] - 0s 31ms/step - loss: 0.2728 - auc: 0.9892
Epoch 94/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2714 - auc: 0.9890
Epoch 95/100
1/1 [==============================] - 0s 27ms/step - loss: 0.2770 - auc: 0.9877
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2770 - auc: 0.9882
Epoch 97/100
1/1 [==============================] - 0s 30ms/step - loss: 0.2864 - auc: 0.9863
Epoch 98/100
1/1 [==============================] - 0s 28ms/step - loss: 

In [9]:
m='remove_Topology'
a=['AtomPairFP',
       'Charge','MAP4','MOE','MorganFP']
b=['xgb','fcnn','lr','dmpnn']
list_auc(a,b)
model.append(m)

Epoch 1/100
1/1 [==============================] - 0s 472ms/step - loss: 3.6585 - auc: 0.2172
Epoch 2/100
1/1 [==============================] - 0s 26ms/step - loss: 2.0715 - auc: 0.3776
Epoch 3/100
1/1 [==============================] - 0s 30ms/step - loss: 1.3173 - auc: 0.7459
Epoch 4/100
1/1 [==============================] - 0s 38ms/step - loss: 1.2347 - auc: 0.8544
Epoch 5/100
1/1 [==============================] - 0s 23ms/step - loss: 1.3506 - auc: 0.8713
Epoch 6/100
1/1 [==============================] - 0s 36ms/step - loss: 1.5048 - auc: 0.8745
Epoch 7/100
1/1 [==============================] - 0s 32ms/step - loss: 1.5711 - auc: 0.8728
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 1.6285 - auc: 0.8705
Epoch 9/100
1/1 [==============================] - 0s 33ms/step - loss: 1.5900 - auc: 0.8743
Epoch 10/100
1/1 [==============================] - 0s 31ms/step - loss: 1.5267 - auc: 0.8848
Epoch 11/100
1/1 [==============================] - 0s 23ms/step - l

1/1 [==============================] - 0s 24ms/step - loss: 0.2710 - auc: 0.9892
Epoch 89/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2687 - auc: 0.9897
Epoch 90/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2655 - auc: 0.9898
Epoch 91/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2590 - auc: 0.9911
Epoch 92/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2755 - auc: 0.9873
Epoch 93/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2608 - auc: 0.9898
Epoch 94/100
1/1 [==============================] - 0s 23ms/step - loss: 0.2562 - auc: 0.9913
Epoch 95/100
1/1 [==============================] - 0s 28ms/step - loss: 0.2563 - auc: 0.9907
Epoch 96/100
1/1 [==============================] - 0s 26ms/step - loss: 0.2616 - auc: 0.9892
Epoch 97/100
1/1 [==============================] - 0s 25ms/step - loss: 0.2637 - auc: 0.9893
Epoch 98/100
1/1 [==============================] - 0s 25ms/step - loss: 

In [10]:
de=pd.DataFrame()
de['model']=model
de['auc']=AUC_ROC
de.to_csv('6feature_remove_one.csv',index=False)
print(de)

               model       auc
0          all_model  0.951983
1  remove_AtomPairFP  0.930276
2      remove_Charge  0.950227
3        remove_MAP4  0.945987
4         remove_MOE  0.951291
5    remove_MorganFP  0.950695
6    remove_Topology  0.950697
